***Syntax natürlicher Sprachen, WS 2022/23***

---
# Übung 11 (Lösung)

In [1]:
import nltk
from collections import defaultdict

---
## Aufgabe 1: Weiterverarbeitung syntaktischer Analysen

#### In dieser Aufgabe sollen Sie die Ausgaben eines state-of-the-art-Parsers, nämlich des spacy-Parsers, weiterverarbeiten.

#### Mit dem Ziel, Sie erst einmal mit den typischen Strukturen einer solchen Aufgabe vertraut zu machen, sollen Sie in dieser Aufgabe lediglich entscheiden, ob die Eingabe einen Infinitivsatz mit Akkusativobjekt enthält.

#### Zur Klarheit betrachten Sie die folgenden positiven und negativen Beispiele:

In [2]:
pos_examples = [
    "Er beabsichtigt , den Kuchen ganz alleine zu essen .",
    "Er behauptet , ihn gesehen zu haben ."
]
neg_examples = [
    "Er glaubt , nach Hause zu fliegen .",
    "Zu fliegen ist schön .",
    "Er will gehen ."
]

#### Zur Erinnerung die wichtigsten Schritte zur Nutzung von spacy:

    1. Modell laden

In [3]:
import spacy
nlp = spacy.load('de_core_news_sm')

    2. Parsen und Visualisieren

In [4]:
for sentence in pos_examples + neg_examples:
    analyzed = nlp(sentence)
    spacy.displacy.render(analyzed, options={'distance':100})

    3. Labels nachschlagen

In [5]:
spacy.explain('oc')

'clausal object'

#### Betrachten Sie die Ausgabe für die Beispielsätze. Schreiben Sie dann eine Funktion, die `True` zurückgibt, wenn ein Satz mit einem Infinitv, der ein Objekt hat, vorliegt und `False` sonst.

In [6]:
def find_accusative(subtree):
    # Hilfsfunktion, die rekursiv Akkusativobjekte sucht
    for child in subtree.children:
        if child.dep_ == 'oa':
            return True
        if child.dep_ == 'oc':
            if find_accusative(child):
                return True
    return False

def classify(sentence):
    # Satz parsen (Syntaxbaum generieren)
    analyzed = nlp(sentence)
    
    # Nach clausal object suchen
    for token in analyzed:
        if token.dep_ == 'oc':
            # wenn gefunden, suche nach Akkusativobjekt
            if find_accusative(token):
                return True

    # wenn Infinitiv in keiner VP nicht gefunden: return False
    return False

Die Ausgabe sollte sein:

```
True
True
False
False
False
```

In [7]:
for p in pos_examples:
    print(classify(p))
for n in neg_examples:
    print(classify(n))

True
True
False
False
False


---
## Aufgabe 2: Informationsextraktion per Syntaxanalyse

#### Gegenstand dieser Aufgabe ist eine anwendungsnahe Möglichkeit, Ergebnisse einer Syntaxanalyse weiterzuverarbeiten. Aus den syntaktischen Abhängigkeiten eines Textes soll (unter Zuhilfenahme einiger Normalisierungsschritte) eine semantische Repräsentation der im Text enthaltenen Informationen gewonnen werden.

#### Für die syntaktische Analyse soll wieder der Dependency Parser von spacy verwendet werden. Die semantische Repräsentation einer Aussage sei ein <a href="https://de.wikipedia.org/wiki/Ontologie_(Informatik)">Knowledge Graph</a> Tripel bestehend aus Subjekt, Prädikat und Objekt (Bei Fehlen von Subjekt oder Objekt soll `None` geschrieben werden.). Die Menge der Prädikate sei durch die Lemmata der vorkommenden Verben definiert. Sie können bei der Implementierung davon ausgehen, dass kein Satz zwei verschiedene Aussagen mit dem gleichen Prädikat enthält.

#### Folgendes Beispiel illustriert das gewünschte Ergebnis:

#### Eingabe:

    I shot an elephant in my pajamas.
    The elephant was seen by a giraffe in the desert.
    The bird I need is a raven.
    The man who saw that raven laughed out loud.

#### Ausgabe:

    (I, shoot, elephant)
    (giraffe, see, elephant)
    (I, need, bird)
    (bird, IS, raven)
    (man, laugh, None)
    (man, see, raven)

In [8]:
sentences = [
    "I shot an elephant in my pajamas.",
    "The elephant was seen by a giraffe in the desert.",
    "The bird I need is a raven.",
    "The man who saw that raven laughed out loud.",
]

In [9]:
nlp = spacy.load('en_core_web_sm')

In [10]:
for sentence in sentences:
    analyzed = nlp(sentence)
    spacy.displacy.render(analyzed, options={'distance':100})
    for token in analyzed:
        print(token.dep_, token.text, token.lemma_)

nsubj I I
ROOT shot shoot
det an an
dobj elephant elephant
prep in in
poss my my
pobj pajamas pajama
punct . .


det The the
nsubjpass elephant elephant
auxpass was be
ROOT seen see
agent by by
det a a
pobj giraffe giraffe
prep in in
det the the
pobj desert desert
punct . .


det The the
nsubj bird bird
nsubj I I
relcl need need
ROOT is be
det a a
attr raven raven
punct . .


det The the
nsubj man man
nsubj who who
relcl saw see
det that that
dobj raven raven
ROOT laughed laugh
advmod out out
advmod loud loud
punct . .


In [11]:
def is_overwritable(token):
    return token is None or token.text in ['who', 'which', 'that', 'whom']

def generate_predicates_for_dep(analyzed):
    predicates = defaultdict(lambda: [None, None])
    
    for token in analyzed:
        if token.dep_ == 'nsubj' and token.head.lemma_ != 'be':
            args = predicates[token.head.lemma_]
            if is_overwritable(args[0]):
                args[0] = token

        if token.dep_ == 'dobj' or token.dep_ == 'nsubjpass':
            args = predicates[token.head.lemma_]
            if is_overwritable(args[1]):
                args[1] = token
        
        if token.dep_ == 'agent':
            for child in token.children:
                if child.dep_ == 'pobj':
                    predicates[token.head.lemma_][0] = child
        
        if token.dep_ == 'attr':
            for sibling in token.head.children:
                if sibling.dep_ == 'nsubj':
                    predicates["IS"][0] = sibling
                    predicates["IS"][1] = token.lemma_
        
        if token.dep_ == 'relcl':
            args = predicates[token.lemma_]
            if is_overwritable(args[0]):
                args[0] = token.head
            elif is_overwritable(args[1]):
                args[1] = token.head
            
    return predicates

def generate_predicates_for_sentence(sentence):    
    analyzed = nlp(sentence)
    predicates = generate_predicates_for_dep(analyzed)
                    
    return [
        "({}, {}, {})".format(elements[0], pred, elements[1])
        for pred, elements in predicates.items()
    ]

In [12]:
for pred in generate_predicates_for_sentence(sentences[3]):
    print(pred)

(man, laugh, None)
(man, see, raven)


In [13]:
def generate_predicates_for_text(text):
    predicates = []
    for sent in text:
        predicates.extend(generate_predicates_for_sentence(sent))
    return predicates

In [14]:
for pred in generate_predicates_for_text(sentences):
    print(pred)

(I, shoot, elephant)
(giraffe, see, elephant)
(I, need, bird)
(bird, IS, raven)
(man, laugh, None)
(man, see, raven)


#### Ideale Ausgabe:

    (I, shoot, elephant)
    (giraffe, see, elephant)
    (I, need, bird)
    (bird, IS, raven)
    (man, laugh, None)
    (man, see, raven)

---
# Hausaufgaben

---
## Aufgabe 3: Mehr Semantik für IE

#### Zusätzlich zu den in Aufgabe 2 behandelten Konstruktionen sollen jetzt auch negierte und komplexe Sätze mit Konjunktionen sinnvoll verarbeitet werden.

#### Eingabe:

    I see an elephant.
    You didn't see the elephant.
    Peter saw the elephant and drank wine.
    
#### Gewünschte Ausgabe:

    (I, see, elephant)
    (You, not_see, elephant)
    (Peter, see, elephant)
    (Peter, drink, wine)
    
#### Kopieren Sie am besten Ihren aktuellen Stand von oben herunter und fügen Sie Ihre Erweiterungen dann hier ein.    

In [15]:
sentences = [
    "I see an elephant.",
    "You didn't see the elephant.",
    "Peter saw the elephant and drank wine."
]

In [16]:
def is_overwritable(token):
    return token is None or token.text in ['who', 'which', 'that', 'whom']

def generate_predicates_for_dep(analyzed):
    predicates = defaultdict(lambda: [None, None])
    negated = set()
    conj = defaultdict(set)
    
    for token in analyzed:
        if token.dep_ == 'nsubj' and token.head.lemma_ != 'be':
            args = predicates[token.head.lemma_]
            if is_overwritable(args[0]):
                args[0] = token
        
        if token.dep_ == 'dobj' or token.dep_ == 'nsubjpass':
            args = predicates[token.head.lemma_]
            if is_overwritable(args[1]):
                args[1] = token
        
        if token.dep_ == 'agent':
            for child in token.children:
                if child.dep_ == 'pobj':
                    predicates[token.head.lemma_][0] = child
        
        if token.dep_ == 'attr':
            for sibling in token.head.children:
                if sibling.dep_ == 'nsubj':
                    predicates["IS-A"][0] = sibling
                    predicates["IS-A"][1] = token.lemma_
        
        if token.dep_ == 'relcl':
            args = predicates[token.lemma_]
            if is_overwritable(args[0]):
                args[0] = token.head
            elif is_overwritable(args[1]):
                args[1] = token.head
                
        if token.dep_ == 'neg':
            negated.add(token.head.lemma_)
            
        if token.dep_ == 'conj':
            conj[token.lemma_].add(token.head.lemma_)
            
    for cc1 in conj:
        for cc2 in conj[cc1]:
            if cc2 in predicates:
                args = predicates[cc1]
                cc_args = predicates[cc2]
                if is_overwritable(args[0]):
                    args[0] = cc_args[0]
                if is_overwritable(args[1]):
                    args[1] = cc_args[1]
            
    for neg in negated:
        predicates['not_' + neg] = predicates[neg]
        del predicates[neg]
            
    return predicates

def generate_predicates_for_sentence(sentence):    
    analyzed = nlp(sentence)
    predicates = generate_predicates_for_dep(analyzed)
                    
    return [
        "({}, {}, {})".format(elements[0], pred, elements[1])
        for pred, elements in predicates.items()
    ]

In [17]:
for pred in generate_predicates_for_text(sentences):
    print(pred)

(I, see, elephant)
(You, not_see, elephant)
(Peter, see, elephant)
(Peter, drank, wine)


#### Ideale Ausgabe:

    (I, see, elephant)
    (You, not_see, elephant)
    (Peter, see, elephant)
    (Peter, drink, wine)